In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('Missing').getOrCreate()

24/08/19 10:53:40 WARN Utils: Your hostname, Shelly-2.local resolves to a loopback address: 127.0.0.1; using 192.168.0.115 instead (on interface en0)
24/08/19 10:53:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/19 10:53:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/19 10:53:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/19 10:53:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/19 10:53:40 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/19 10:53:40 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [4]:
training=spark.read.csv('tt4.csv',inferSchema=True,header=True)
training.show()

+----+---+---+------+
|Name|Age|Exp|Salary|
+----+---+---+------+
|   A| 11|  1|   100|
|   B| 23|  2|   200|
|   C| 45|  3|   300|
|   D| 12|  1|   400|
|   E|  3|  2|   500|
+----+---+---+------+



In [5]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Exp: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [6]:
training.columns

['Name', 'Age', 'Exp', 'Salary']

In [7]:
#Group the data into a vector
from pyspark.ml.feature import VectorAssembler

In [10]:
featureassembler=VectorAssembler(
    inputCols=['Age', 'Exp'],
    outputCol="Independent Features"
)
#create new column with vector values

In [11]:
output=featureassembler.transform(training)

In [12]:
output.show()

+----+---+---+------+--------------------+
|Name|Age|Exp|Salary|Independent Features|
+----+---+---+------+--------------------+
|   A| 11|  1|   100|          [11.0,1.0]|
|   B| 23|  2|   200|          [23.0,2.0]|
|   C| 45|  3|   300|          [45.0,3.0]|
|   D| 12|  1|   400|          [12.0,1.0]|
|   E|  3|  2|   500|           [3.0,2.0]|
+----+---+---+------+--------------------+



In [14]:
finalised_data=output.select("Independent Features","Salary")
finalised_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [11.0,1.0]|   100|
|          [23.0,2.0]|   200|
|          [45.0,3.0]|   300|
|          [12.0,1.0]|   400|
|           [3.0,2.0]|   500|
+--------------------+------+



In [15]:
from pyspark.ml.regression import LinearRegression

In [16]:
train_data, test_data=finalised_data.randomSplit([0.75,0.25])

In [17]:
regressor=LinearRegression(
    featuresCol='Independent Features',
    labelCol='Salary'
)

regressor=regressor.fit(train_data)

24/08/19 11:15:33 WARN Instrumentation: [033d39e2] regParam is zero, which might cause numerical instability and overfitting.
24/08/19 11:15:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/19 11:15:33 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/08/19 11:15:34 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [18]:
regressor.coefficients

DenseVector([-6.8143, 62.4361])

In [19]:
regressor.intercept

366.5247018739351

In [20]:
pred_results=regressor.evaluate(test_data)

In [22]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [11.0,1.0]|   100|354.00340715502546|
+--------------------+------+------------------+



In [23]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

(254.00340715502546, 64517.730846361635)

24/08/20 18:59:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 2710807 ms exceeds timeout 120000 ms
24/08/20 18:59:52 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/20 19:00:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$